# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

*Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?*

This is a classification problem, as we are trying to classify a given student as 'needing intervention' or 'not needing intervention' - assigning him/her to a discrete category.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

from IPython.core.display import display, HTML

HTML('''<style>
        table, td {
            width: 100%;
            table-layout: fixed;
            text-align: right;
        }
        
    </style>''')

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
#print student_data.shape
#print list(student_data.columns.values)

n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1 #Need to exclude label column from the table
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = n_students - n_passed
grad_rate = float(n_passed) / float(n_students) * 100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
from sklearn.cross_validation import StratifiedShuffleSplit
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train
test_percentage = 1 - (num_train / float(num_all))

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

X = np.array(X_all)
y = np.array(y_all)

FOLDS = 10
strat_shuffle = StratifiedShuffleSplit(y, FOLDS, test_size = test_percentage, random_state = 9)

for train_index, test_index in strat_shuffle:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# # Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
***
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

Answer:

For classifying students requiring intervention, I chose to train and test using the following models:
- Support Vector Machines (svm.SVC())
- AdaBoost (ensemble.AdaBoostClassifier())
- K-Nearest Neighbors (neighbors.KNeighborsClassifier())

<table width="100%">
<tr>
<th>Model</th>
<th>Support Vector Machines</th>
<th>AdaBoost</th>
<th>K-Nearest Neighbors</th>
</tr>
<tr>
<td>Theoretical O(n) Time Complexity</td>
<td>O(n) to O(n<sup>3</sup>)</td>
<td>O(n, D<sup>2</sup>)</td>
<td>Training: O(1) | Query: O(log(n) + k)</td>
</tr>
<tr>
<td>Theoretical O(n) Space Complexity</td>
<td>O(n, D)</td>
<td>O(n log n) for a kd-tree median of medians algorithm</td>
<td>Training: n | Query: 1</td>
</tr>
<tr>
<td>Strengths</td>
<td>Use of a linear kernel (as in this case) and adjustments to C parameter values help avoid overfitting, classifier is memory efficent</td>
<td>Predictive power robust to variety of parameter values and classifier types, not very susceptible to overfitting (if underlying classifier not overfitting)</td>
<td>No cost of learning, no functional assumptions required, only works well if all features matter equally</td>
</tr>
<tr>
<td>Weaknesses</td>
<td>Extremely slow in training, performance highly sensitive to kernel function choice</td>
<td>Can be sensitive to noisy data and outliers, may overfit if noise is uniformly distributed or underlying classifiers overfit</td>
<td>Expensive and slow in testing - requires storing all training data points</td>
</tr>
<tr>
<td>Why chosen for classifying available data?</td>
<td>Problem doesn't have training time restrictions, SVM useful where model has large number of features</td>
<td>Good general-purpose classifier - I didn't have a good idea of what parameter values for model would be appropriate</td>
<td>Wanted to compare performance of non-parametric model, don't have constraints on data storage</td>
</tr>
<td colspan="4">
See below for F1 outcomes
Note: n = number of elements in training set, D = number of features in model, k = number of nearest neighbors used in k-nn classification
</td>
</tr>
</table>


In [14]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
import sklearn
from sklearn import svm, tree, neighbors, ensemble
#Use clf for SVC classifier in this case - to generalize later for output tables
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.005
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [15]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Prediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Prediction time (secs): 0.004
F1 score for training set: 0.872807017544


In [16]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Prediction time (secs): 0.002
F1 score for test set: 0.778523489933


In [17]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
    return predict_labels(clf, X_test, y_test)

NEIGHBORS = 15

sv_clf = svm.SVC(kernel='linear', C=100)
ad_clf = ensemble.AdaBoostClassifier(n_estimators=10)
kn_clf = neighbors.KNeighborsClassifier(n_neighbors = NEIGHBORS, weights = "distance")
    
classifiers = [sv_clf, ad_clf, kn_clf] 

##Building datasets for 3 different training dataset sizes
training_set_sizes = [100, 200, 300]
X_train_100, X_train_200, X_train_300 = [],[],[]
y_train_100, y_train_200, y_train_300 = [],[],[]
X_training_sets = [X_train_100, X_train_200, X_train_300]
y_training_sets = [y_train_100, y_train_200, y_train_300]

#Ensuring array data still accessible
X = np.array(X_all)
y = np.array(y_all)

for idx, size in enumerate(training_set_sizes):
    strat_shuffle = StratifiedShuffleSplit(y, test_size = 95, train_size = size, random_state = 9)
    for train_index, test_index in strat_shuffle:
        X_training_sets[idx], X_test = X[train_index], X[test_index]
        y_training_sets[idx], y_test = y[train_index], y[test_index]

#Example loop
# for classifier in classifiers:
#     for idx in range(0,len(training_set_sizes)):
#         train_predict(classifier, X_training_sets[idx], y_training_sets[idx], X_test, y_test)
        
def table_row_output():
    row_titles = ['100-element training set','200-element training set','300-element training set']
    columns = [c.__class__.__name__ for c in classifiers]
    a = {}
    for classifier in classifiers:
        a[classifier.__class__.__name__] = []
        for idx in range(0,len(training_set_sizes)):
            a[classifier.__class__.__name__].append(format(round(train_predict(classifier, X_training_sets[idx], y_training_sets[idx], X_test, y_test),3)))
        a[classifier.__class__.__name__] = np.array(a[classifier.__class__.__name__])
    display(HTML("<h2>" + "Test Set F1 Score Table for Selected Classifiers" + "</h2>")) 
    return pd.DataFrame(a, index=row_titles, columns=columns).to_html()

HTML(table_row_output())



------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.057
Predicting labels using SVC...
Prediction time (secs): 0.000
F1 score for training set: 0.985294117647
Predicting labels using SVC...
Prediction time (secs): 0.000
F1 score for test set: 0.769230769231
Predicting labels using SVC...
Prediction time (secs): 0.000
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 2.639
Predicting labels using SVC...
Prediction time (secs): 0.001
F1 score for training set: 0.864111498258
Predicting labels using SVC...
Prediction time (secs): 0.001
F1 score for test set: 0.806201550388
Predicting labels using SVC...
Prediction time (secs): 0.000
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 3.018
Predicting labels using SVC...
Prediction time (secs): 0.002
F1 score for training set: 0.851351351351
Predicting labels using

,SVC,AdaBoostClassifier,KNeighborsClassifier
100-element training set,0.769,0.769,0.824
200-element training set,0.806,0.754,0.894
300-element training set,0.761,0.763,0.784


All of the models provided provide a similar (reasonably high) level of performance, using F1 score as our metric, with K-Nearest Neighbors providing the highest scores among the three classifiers for each value for the size of our training data set.


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?


Based on the experiment above, I would use a K-Nearest Neighbors classifier to identify students who may require intervention. Of the analyzed classifiers, it seems shows the highest predictive power at each training set size.

Besides showing the highest predictive power it also requires far less training time than the SVC model (where training lasts 2.9 seconds for the 300-element set) and somewhat less time than the AdaBoost model, as we would theoretically expect, because K-Nearest Neighbors doesn't require training at all.

Prediction time for K-Nearest Neighbors is also not significantly greater than for the other two classifiers, clocking in at 0.001 seconds for the 300-element test set.

One potential downside to K-Nearest Neighbors could be the storage capacity required, as all training set data must be retained and queried against for each KNN test prediction, but given the relatively small size of our dataset, this doesn't seem to prompt material concerns. 

## Support Vector Machines
### Classifier Performance Overview

<table class="c10">
<tbody>
    <tr class="c9">
        <td class="c8 c11" colspan="1" rowspan="2">
            <p class="c1">
            <span class="c0">
            </span>
            </p>
        </td>
        <td class="c8 c12" colspan="3" rowspan="1">
            <p class="c6">
            <span class="c0">Training set size</span>
            </p>
        </td>
     </tr>
     <tr class="c9">
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">100</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">200</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">300</span>
             </p>
         </td>
      </tr><tr class="c5">
          <td class="c11" colspan="1" rowspan="1">
              <p class="c6">
              <span class="c0">Training time (secs)</span>
              </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.059</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1">
               <span class="c0">2.690</span>
               </p>
               </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">2.926</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">Prediction time (secs)</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">F1 score for training set</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.985</span>
               </p></td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.864</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.851</span></p>
           </td>
        </tr>
        <tr class="c5", style="font-weight:bold">
            <td class="c11" colspan="1" rowspan="1">
                <p class="c6"><span class="c0">F1 score for test set</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.769</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.806</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.761</span></p>
            </td>
         </tr>
     </tbody>
</table>

## AdaBoost
### Classifier Performance Overview

<table class="c10">
<tbody>
    <tr class="c9">
        <td class="c8 c11" colspan="1" rowspan="2">
            <p class="c1">
            <span class="c0">
            </span>
            </p>
        </td>
        <td class="c8 c12" colspan="3" rowspan="1">
            <p class="c6">
            <span class="c0">Training set size</span>
            </p>
        </td>
     </tr>
     <tr class="c9">
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">100</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">200</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">300</span>
             </p>
         </td>
      </tr><tr class="c5">
          <td class="c11" colspan="1" rowspan="1">
              <p class="c6">
              <span class="c0">Training time (secs)</span>
              </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.015</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1">
               <span class="c0">0.015</span>
               </p>
               </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.015</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">Prediction time (secs)</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">F1 score for training set</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.861</span>
               </p></td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.825</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.849</span></p>
           </td>
        </tr>
        <tr class="c5", style="font-weight:bold">
            <td class="c11" colspan="1" rowspan="1">
                <p class="c6"><span class="c0">F1 score for test set</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.769</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.754</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.763</span></p>
            </td>
         </tr>
     </tbody>
</table>

## K-Nearest Neighbors
### Classifier Performance Overview

<table class="c10">
<tbody>
    <tr class="c9">
        <td class="c8 c11" colspan="1" rowspan="2">
            <p class="c1">
            <span class="c0">
            </span>
            </p>
        </td>
        <td class="c8 c12" colspan="3" rowspan="1">
            <p class="c6">
            <span class="c0">Training set size</span>
            </p>
        </td>
     </tr>
     <tr class="c9">
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">100</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">200</span>
             </p>
         </td>
         <td class="c7 c8" colspan="1" rowspan="1">
             <p class="c6">
             <span class="c0">300</span>
             </p>
         </td>
      </tr><tr class="c5">
          <td class="c11" colspan="1" rowspan="1">
              <p class="c6">
              <span class="c0">Training time (secs)</span>
              </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.000</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1">
               <span class="c0">0.000</span>
               </p>
               </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.000</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">Prediction time (secs)</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.001</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.003</span>
               </p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">0.005</span>
               </p>
           </td>
       </tr>
       <tr class="c5">
           <td class="c11" colspan="1" rowspan="1">
               <p class="c6"><span class="c0">F1 score for training set</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">1.0</span>
               </p></td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">1.0</span></p>
           </td>
           <td class="c7" colspan="1" rowspan="1">
               <p class="c1"><span class="c0">1.0</span></p>
           </td>
        </tr>
        <tr class="c5" , style="font-weight:bold">
            <td class="c11" colspan="1" rowspan="1">
                <p class="c6"><span class="c0">F1 score for test set</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.824</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.894</span></p>
            </td>
            <td class="c7" colspan="1" rowspan="1">
                <p class="c1"><span class="c0">0.784</span></p>
            </td>
         </tr>
     </tbody>
</table>

## Optimizing K-Nearest Neighbors Model



In [40]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer 

f1_scorer = make_scorer(f1_score, pos_label="yes")

cv = StratifiedShuffleSplit(y_train, 30, test_size = test_percentage, random_state = 9)

print "The original {} model used the {} algorithm to classify examples \n".format(kn_clf.__class__.__name__,kn_clf.algorithm)

optimized_clf = GridSearchCV(neighbors.KNeighborsClassifier(),
                   param_grid={"n_neighbors": [5,7,9,11,13,15,17,19,21,23,25],
                               "weights": ['uniform', 'distance'],
                               "algorithm": ['ball_tree', 'kd_tree', 'brute']}, scoring = f1_scorer, cv=cv)

#Uses the full training set (300 data points) for X and y
optimized_clf.fit(X_train,y_train)
best_param_dict = optimized_clf.best_params_

for param in best_param_dict:
    print "The optimal value for parameter {} is {}".format(param, best_param_dict[param]) 

best_param_clf = neighbors.KNeighborsClassifier(n_neighbors = best_param_dict['n_neighbors'], weights = "uniform", 
                                                    algorithm = 'brute')

best_param_clf.fit(X_train, y_train)

print "\nF1 score for classifier with optimized parameters on test set: {}".format(round(predict_labels(best_param_clf, 
                                                                                                      X_test, y_test),3))


The original KNeighborsClassifier model used the auto algorithm to classify examples 

The optimal value for parameter n_neighbors is 19
The optimal value for parameter weights is uniform
The optimal value for parameter algorithm is brute
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.001

F1 score for classifier with optimized parameters on test set: 0.8


After running a grid search algorithm cross-validated with StratifiedShuffleSplit, I find that the optimal model uses a parameter value of 19 for `n_neighbors`, uniform weighting, and the brute force algorithm for classification.

The adjustment to the original K-Nearest Neighbor parameters (where we originally used a value of 15 for `n_neighbors`, `distance` weighting and the default `auto` algorithm) provides a model achieving a F1 score of 0.8 when run on our test set.

To summarize the K-Nearest Neighbors classifier in layman's terms I would say that this classification approach identifies a student that needs intervention by seeing if students with similar characteristics have typically failed. 

The process of classifying a student with K-Nearest Neighbors (in our optimized model) runs in the following way:

1. We find the 19 students that are most 'similar' to the student, based on all the 30 features we're analyzing
2. We look at whether more of those similar students failed or passed
3. If more students from this group of 19 have failed (than not), then we classify the student as at risk for failing, and assign him/her to a 'needs intervention' group

To help explain what we mean by 'similar', imagine that we are only comparing students for this classifier using 2 characteristics - student age and GPA. If we put a point for each student on a map where the point distance on the y-axis is based on GPA and the point distance on the x-axis is based on GPA, then the most 'similar' students would be those located the shortest straight-line distance away from our studied student.

For gauging similarity in our actual process, we are doing essentially the same thing as in the 2D map comparison described, but measuring the student's distance to other students in a space with 30 dimensions. Once we find the 19 closest students, then we simply 'vote' to decide whether the student is likely to fail or not, based on what these 11 closest students have done.

### References:

On Algorithm Complexity -
- Support Vector Machines
http://scikit-learn.org/stable/modules/svm.html#complexity
http://stats.stackexchange.com/questions/94596/computational-complexity-of-prediction-using-svm-and-nn
http://stackoverflow.com/questions/16585465/training-complexity-of-linear-svm
http://www.csie.ntu.edu.tw/~cjlin/papers/libsvm.pdf
http://stats.stackexchange.com/questions/96995/machine-learning-classifiers-big-o-or-complexity
http://machinelearning.wustl.edu/mlpapers/paper_files/TsangKC05.pdf
http://stats.stackexchange.com/questions/35276/svm-overfitting-curse-of-dimensionality


- Adaboost
http://stackoverflow.com/questions/22397485/what-is-the-o-runtime-complexity-of-adaboost


- KNN Complexity
https://classroom.udacity.com/nanodegrees/nd009/parts/0091345404/modules/544698886575460/lessons/5396240646/concepts/6727188230923


Other topics

https://en.wikipedia.org/wiki/K-d_tree
http://www.mzan.com/article/34221712-grid-search-with-f1-as-scoring-function-several-pages-of-error-message.shtml#sthash.ymcXZdjq.dpuf